## performing multivariate time series analysis using sklearn

youtube link : https://www.youtube.com/watch?v=RRd2wzMRpOc

In [1]:
import pandas as pd  
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('./Sales_Transactions_Dataset_Weekly.csv')
df = df.filter(regex = r'Product|W')
df.head()

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
0,P1,11,12,10,8,13,12,14,21,6,...,4,7,8,10,12,3,7,6,5,10
1,P2,7,6,3,2,7,1,6,3,3,...,2,4,5,1,1,4,5,1,6,0
2,P3,7,11,8,9,10,8,7,13,12,...,6,14,5,5,7,8,14,8,8,7
3,P4,12,8,13,5,9,6,9,13,13,...,9,10,3,4,6,8,14,8,7,8
4,P5,8,5,13,11,6,7,9,14,9,...,7,11,7,12,6,6,5,11,8,9


this data contains weekly sales data for each product

In [3]:
# stacking the data
melt = df.melt(id_vars = 'Product_Code', var_name = 'Week', value_name = 'Sales')
melt.head()

,Product_Code,Week,Sales
0,P1,W0,11
1,P2,W0,7
2,P3,W0,7
3,P4,W0,12
4,P5,W0,8


In [4]:
melt['Product_Code'] = melt['Product_Code'].str.extract('(\d+)', expand=False).astype(int)
melt['Week'] = melt['Week'].str.extract('(\d+)', expand=False).astype(int)

melt = melt.sort_values(['Week', 'Product_Code'])
melt.head()

,Product_Code,Week,Sales
0,1,0,11
1,2,0,7
2,3,0,7
3,4,0,12
4,5,0,8


In [5]:
# splitting the data
split_point = 40
melt_train = melt[melt.Week < split_point].copy()
melt_valid = melt[melt.Week >= split_point].copy()

In [6]:
# now our goal is the predict the sales values for next week, so we will create a new column in the dataframe that will have next week's sales values
melt_train['sales_next_week'] = melt_train.groupby("Product_Code")['Sales'].shift(-1)   #we shift by 1 because we want to predict next week values

# if you want to predict 4 weeks of sales, then shift the values by -4


In [7]:
melt_train[melt_train['Product_Code'] == 1].head()  

,Product_Code,Week,Sales,sales_next_week
0,1,0,11,12.0
811,1,1,12,10.0
1622,1,2,10,8.0
2433,1,3,8,13.0
3244,1,4,13,12.0


you can notice that sales_next_week values are next week's sales values(see sales and sales_next_week features)

In [8]:
melt_train.tail()

,Product_Code,Week,Sales,sales_next_week
32435,815,39,2,NaN
32436,816,39,6,NaN
32437,817,39,0,NaN
32438,818,39,0,NaN
32439,819,39,0,NaN


since training data doesn't have next week's data, the values are nan

In [9]:
melt_train = melt_train.dropna()
melt_train.tail()

,Product_Code,Week,Sales,sales_next_week
31624,815,38,1,2.0
31625,816,38,4,6.0
31626,817,38,0,0.0
31627,818,38,0,0.0
31628,819,38,1,0.0


In [10]:
# performing the same operation on validation data
melt_valid['sales_next_week'] = melt_valid.groupby("Product_Code")['Sales'].shift(-1)

In [11]:
melt_valid.head()

,Product_Code,Week,Sales,sales_next_week
32440,1,40,7,11.0
32441,2,40,2,5.0
32442,3,40,3,5.0
32443,4,40,12,6.0
32444,5,40,5,8.0


In [12]:
# create a lag for the data. the sales values will be closest to the past week so we will take lag = 1. you can also use last year's same week values but we dont have the data
melt_train["lag_sales_1"] = melt_train.groupby("Product_Code")['Sales'].shift(1)

In [13]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1
0,1,0,11,12.0,NaN
811,1,1,12,10.0,11.0
1622,1,2,10,8.0,12.0
2433,1,3,8,13.0,10.0
3244,1,4,13,12.0,8.0


lag_sales_1 will contain past week sales value.

we now have sales_next_week that contains sales values for next week and lag_sales_1 for past week sales values

In [14]:
# performing the same operation on validation data
melt_valid["lag_sales_1"] = melt_valid.groupby("Product_Code")['Sales'].shift(1)

In [15]:
melt_valid[melt_valid.Product_Code == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1
32440,1,40,7,11.0,NaN
33251,1,41,11,4.0,7.0
34062,1,42,4,7.0,11.0
34873,1,43,7,8.0,4.0
35684,1,44,8,10.0,7.0


In [16]:
# differencing
melt_train["diff_sales_1"] = melt_train.groupby("Product_Code")['Sales'].diff(1)

In [17]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1
0,1,0,11,12.0,NaN,NaN
811,1,1,12,10.0,11.0,1.0
1622,1,2,10,8.0,12.0,-2.0
2433,1,3,8,13.0,10.0,-2.0
3244,1,4,13,12.0,8.0,5.0


In [18]:
# performing the same operation on validation data
melt_valid["diff_sales_1"] = melt_valid.groupby("Product_Code")['Sales'].diff(1)

In [19]:
melt_valid[melt_valid.Product_Code == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1
32440,1,40,7,11.0,NaN,NaN
33251,1,41,11,4.0,7.0,4.0
34062,1,42,4,7.0,11.0,-7.0
34873,1,43,7,8.0,4.0,3.0
35684,1,44,8,10.0,7.0,1.0


In [20]:
# rolling statistics
melt_train["mean_sales_4"] = melt_train.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)

In [21]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4
0,1,0,11,12.0,NaN,NaN,NaN
811,1,1,12,10.0,11.0,1.0,NaN
1622,1,2,10,8.0,12.0,-2.0,NaN
2433,1,3,8,13.0,10.0,-2.0,10.25
3244,1,4,13,12.0,8.0,5.0,10.75


In [22]:
# rolling statistics on validation data
melt_valid["mean_sales_4"] = melt_valid.groupby("Product_Code")['Sales'].rolling(4).mean().reset_index(level=0, drop=True)
melt_valid[melt_valid.Product_Code == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4
32440,1,40,7,11.0,NaN,NaN,NaN
33251,1,41,11,4.0,7.0,4.0,NaN
34062,1,42,4,7.0,11.0,-7.0,NaN
34873,1,43,7,8.0,4.0,3.0,7.25
35684,1,44,8,10.0,7.0,1.0,7.50


# 4.4 Date components (seasonality) - not available in this dataset
- Day of week
- Day of year
- Month
- Day of month

### Extra tip: what about interactions? (e.g.: Lag 1 - Rolling 7)

### selecting a evaluation metric  :
 most common are :
 
 -> MAPE(mean absolute percentage error) 

 ![alt text](mape.png)
 
 -> WMAPE(weighted mean absolute percentage error)

 ![alt text](wmape.png)
 
the drawback for MAPE is that sometimes there might be occurance of divide by zero error so to avoid that we use WMAPE

In [23]:
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    #ape[~np.isfinite(ape)] = 0. # VERY questionable
    ape[~np.isfinite(ape)] = 1. # pessimist estimate
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

In [24]:
# establishing a baseline performance : we will use this number as a baseline number to evaluate our model's performance.(you can also use other model's accuracy(if your company already has one) as the baseline number for this model). but here, we don't have any other model we will just use the error between this week's sales and sales_next_week values as baseline performance

ypred = melt_train['Sales']
ytrue = melt_train['sales_next_week']

In [25]:
mape(ytrue, ypred)

0.6721872645511404

In [26]:
wmape(ytrue, ypred)

0.30816465612331645

In [27]:
melt_train.columns

Index(['Product_Code', 'Week', 'Sales', 'sales_next_week', 'lag_sales_1',
       'diff_sales_1', 'mean_sales_4'],
      dtype='object')

In [28]:
features = ['Sales','lag_sales_1','diff_sales_1', 'mean_sales_4']

# imputing the Nan
imputer = SimpleImputer()
Xtr = imputer.fit_transform(melt_train[features])
ytr = melt_train['sales_next_week']


# training the model
mdl = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
mdl.fit(Xtr, ytr)
# try hyperparameter tuning the model

RandomForestRegressor(n_jobs=6, random_state=0)

In [29]:
# model evaluation
Xval = imputer.transform(melt_valid[features])
yval = melt_valid['sales_next_week']

p = mdl.predict(Xval)

In [30]:
mape(yval, p)

0.6463137461455405

In [31]:
wmape(yval, p)

0.3004969729507602

In [32]:
# extending the model to predict 'n' weeks
# we will now perform for next 2 weeks

melt_train['sales_next_next_week'] = melt_train.groupby("Product_Code")['Sales'].shift(-2)
melt_valid['sales_next_next_week'] = melt_valid.groupby("Product_Code")['Sales'].shift(-2)

In [33]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
0,1,0,11,12.0,NaN,NaN,NaN,10.0
811,1,1,12,10.0,11.0,1.0,NaN,8.0
1622,1,2,10,8.0,12.0,-2.0,NaN,13.0
2433,1,3,8,13.0,10.0,-2.0,10.25,12.0
3244,1,4,13,12.0,8.0,5.0,10.75,14.0


In [34]:
melt_train = melt_train.dropna(subset=['sales_next_week','sales_next_next_week'])

In [35]:
imputer = SimpleImputer()
Xtr = imputer.fit_transform(melt_train[features])
ytr = melt_train[['sales_next_week', 'sales_next_next_week']]

mdl = RandomForestRegressor(n_estimators=100, random_state=0, n_jobs=6)
mdl.fit(Xtr, ytr)

RandomForestRegressor(n_jobs=6, random_state=0)

In [36]:
Xval = imputer.transform(melt_valid[features])
yval = melt_valid[['sales_next_week', 'sales_next_next_week']]

p = mdl.predict(Xval)

In [37]:
mape(yval, p)

0.6640898798348054

In [38]:
wmape(yval, p)

c:\Users\RAJESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


sales_next_week         0.300301
sales_next_next_week    0.310315
dtype: float64

# 10. Predicting new examples
- As long as you have the same features you used to train, you can predict for any period

In [39]:
# predicting new examples
melt_valid.tail()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
42167,815,51,0,NaN,2.0,-2.0,0.50,NaN
42168,816,51,5,NaN,6.0,-1.0,5.25,NaN
42169,817,51,3,NaN,4.0,-1.0,1.75,NaN
42170,818,51,0,NaN,2.0,-2.0,0.50,NaN
42171,819,51,1,NaN,0.0,1.0,0.25,NaN


In [40]:
new_examples = melt_valid[melt_valid['Week'] == 51].copy()
new_examples.head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
41361,1,51,10,NaN,5.0,5.0,7.00,NaN
41362,2,51,0,NaN,6.0,-6.0,3.00,NaN
41363,3,51,7,NaN,8.0,-1.0,9.25,NaN
41364,4,51,8,NaN,7.0,1.0,9.25,NaN
41365,5,51,9,NaN,8.0,1.0,8.25,NaN


In [41]:
p = mdl.predict(new_examples[features])
p

c:\Users\RAJESH\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


array([[ 4.70014286,  8.85766667],
       [ 1.69609452,  3.01583297],
       [10.90119048,  5.80107143],
       ...,
       [ 0.74560423,  2.15993741],
       [ 0.52880028,  0.49030189],
       [ 0.27852871,  0.30830163]])

In [42]:
new_examples['p_sales_next_week'] = p[:, 0]
new_examples['p_sales_next_next_week'] = p[:, 1]

In [43]:
new_examples.head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week,p_sales_next_week,p_sales_next_next_week
41361,1,51,10,NaN,5.0,5.0,7.00,NaN,4.700143,8.857667
41362,2,51,0,NaN,6.0,-6.0,3.00,NaN,1.696095,3.015833
41363,3,51,7,NaN,8.0,-1.0,9.25,NaN,10.901190,5.801071
41364,4,51,8,NaN,7.0,1.0,9.25,NaN,8.492500,13.952071
41365,5,51,9,NaN,8.0,1.0,8.25,NaN,9.381560,8.619524
